In [16]:
import pandas as pd
from datetime import datetime, timedelta, time
import json
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [17]:
today_date = datetime.today().date()
today_date

datetime.date(2024, 12, 9)

# Preparing Emp Availability With Working Flag and Time Remaining

In [61]:
# Testing
# Read the data
path="00_Input/01_Emp_Availability_Initial.xlsx"
df = pd.read_excel(path, names=['Name', 'Responsibility', 'Time in', 'Time out'])
df.head()

path="00_Input/01_Emp_Availability_Initial_csv.csv"
df = pd.read_csv(path, names=['Name', 'Responsibility', 'Time in', 'Time out'])
df.head()
df.iloc[0,3]
# df['Time in'] = pd.to_datetime(df['Time in'], format='%I:%M:%S %p').apply(lambda x: datetime.combine(today_date, x.time()))
# df['Time out'] = pd.to_datetime(df['Time out'], format='%I:%M:%S %p').apply(lambda x: datetime.combine(today_date, x.time()))
# df.head()

'10:00:00 AM'

In [66]:
df.columns
print(df.iloc[1,])

Name              Aidan Priller
Responsibility      Sales Floor
Time in              8:45:00 AM
Time out            10:00:00 AM
Name: 1, dtype: object


## 1. Collect Emp Availability Table and Prepare

In [ ]:
# Read the data
path="00_Input/01_Emp_Availability_Initial.xlsx"
df = pd.read_excel(path, names=['Name', 'Responsibility', 'Time in', 'Time out'])

# Convert 'Time in' and 'Time out' to datetime with today's date
df['Time in'] = pd.to_datetime(df['Time in'], format='%Y-%m-%d %H:%M:%S').apply(lambda x: datetime.combine(today_date, x.time()))
df['Time out'] = pd.to_datetime(df['Time out'], format='%I:%M:%S %p').apply(lambda x: datetime.combine(today_date, x.time()))

# Note: If the initial shift allocation starts (Time In) at any 15mins interval (like 8:45), we are shifting to later 30mins (9:00) AND if the initial shift allocation ends (Time Out) at any 15mins interval (like 6:15), we are shifting to previous 30mins (6:00). 
# Function to adjust times based on 15-minute intervals
def adjust_time_in(time_in):
    if time_in.minute == 45:
        # Shift forward to the next 30-minute mark
        return time_in.replace(minute=0) + timedelta(hours=1)
    elif time_in.minute == 30:
        # Already at a 30-minute mark, do nothing
        return time_in
    elif time_in.minute == 15:
        # Shift forward to the next 30-minute mark
        return time_in.replace(minute=30)
    return time_in

def adjust_time_out(time_out):
    if time_out.minute == 15:
        # Shift backward to the previous 30-minute mark
        return time_out.replace(minute=0)
    elif time_out.minute == 30:
        # Already at a 30-minute mark, do nothing
        return time_out
    elif time_out.minute == 45:
        # Shift backward to the previous 30-minute mark
        return time_out.replace(minute=30)
    return time_out

# Apply adjustments to the 'Time in' and 'Time out' columns
df['Time in'] = df['Time in'].apply(adjust_time_in)
df['Time out'] = df['Time out'].apply(adjust_time_out)

# Filter out not-required roles and names
filtered_df = df[~df['Responsibility'].isin(['Technology', 'Office Work']) & ~df['Name'].str.contains('Available')]

# Display the filtered DataFrame
filtered_df.head(5)


,Name,Responsibility,Time in,Time out
0,Ana* Gonzalez,Lead Student,2024-12-09 08:30:00,2024-12-09 10:00:00
1,Aidan Priller,Sales Floor,2024-12-09 09:00:00,2024-12-09 10:00:00
2,Analuisa Flores Teran,Sales Floor,2024-12-09 09:00:00,2024-12-09 11:00:00
3,Caroline Lester,Sales Floor,2024-12-09 09:00:00,2024-12-09 10:00:00
4,Carson Turk,Greeter,2024-12-09 09:00:00,2024-12-09 13:00:00


In [19]:
# Testing
filtered_df[filtered_df['Name']=='Ana* Gonzalez']

,Name,Responsibility,Time in,Time out
0,Ana* Gonzalez,Lead Student,2024-12-09 08:30:00,2024-12-09 10:00:00


## 2. Create full joined working flag table

In [20]:
new_rows = []
time_interval=30

first_start_time = filtered_df['Time in'].min()
max_end_time = filtered_df['Time out'].max()

# Get unique names from filtered_df
unique_names = filtered_df['Name'].unique()

# Iterate through each individual
for name in unique_names:
    start_time = first_start_time
    end_time = max_end_time
    
    # Generate 30 minute intervals
    while start_time < end_time:
        new_row = {
            'Name': name,
            'Start_time': start_time.time(),
            'End_time': (start_time + timedelta(minutes=time_interval)).time()
        }
        new_rows.append(new_row)
        start_time += timedelta(minutes=30)

# Create the new DataFrame
work_status_df = pd.DataFrame(new_rows)


def get_working_flag(name, start_time, end_time):
    # Get all working hours for the specific name
    employee_records = filtered_df[filtered_df['Name'] == name]
    
    # Iterate through all records for the employee
    for index, record in employee_records.iterrows():
        # Convert Time in and Time out to time objects
        time_in = record['Time in'].time()  # Use .time() to get the time object
        time_out = record['Time out'].time()  # Use .time() to get the time object

        # Check if the Start_time and End_time fall within the working hours
        if (time_in <= start_time < time_out) or (time_in < end_time <= time_out) or (start_time <= time_in and end_time >= time_out):
            return 1  # Working
    
    return 0  # Not working if none of the records match

# Add the 'Working Flag' column to work_status_df
work_status_df['Working Flag'] = work_status_df.apply(
    lambda row: get_working_flag(row['Name'], row['Start_time'], row['End_time']), axis=1
)

# Display the updated DataFrame
# work_status_df.sample(5)
work_status_df.head(15)

,Name,Start_time,End_time,Working Flag
0,Ana* Gonzalez,08:30:00,09:00:00,1
1,Ana* Gonzalez,09:00:00,09:30:00,1
2,Ana* Gonzalez,09:30:00,10:00:00,1
3,Ana* Gonzalez,10:00:00,10:30:00,0
4,Ana* Gonzalez,10:30:00,11:00:00,0
5,Ana* Gonzalez,11:00:00,11:30:00,0
6,Ana* Gonzalez,11:30:00,12:00:00,0
7,Ana* Gonzalez,12:00:00,12:30:00,0
8,Ana* Gonzalez,12:30:00,13:00:00,0
9,Ana* Gonzalez,13:00:00,13:30:00,0


In [21]:
# Testing 
work_status_df[work_status_df['Name']=='Ana* Gonzalez']

,Name,Start_time,End_time,Working Flag
0,Ana* Gonzalez,08:30:00,09:00:00,1
1,Ana* Gonzalez,09:00:00,09:30:00,1
2,Ana* Gonzalez,09:30:00,10:00:00,1
3,Ana* Gonzalez,10:00:00,10:30:00,0
4,Ana* Gonzalez,10:30:00,11:00:00,0
5,Ana* Gonzalez,11:00:00,11:30:00,0
6,Ana* Gonzalez,11:30:00,12:00:00,0
7,Ana* Gonzalez,12:00:00,12:30:00,0
8,Ana* Gonzalez,12:30:00,13:00:00,0
9,Ana* Gonzalez,13:00:00,13:30:00,0


In [22]:
# Caluculate remaining hours left 

work_status_df= work_status_df[work_status_df['Working Flag']==1]
greeter_priority_df= work_status_df.copy()
work_status_copy_df= work_status_df.copy()

# Calculate remaining hours left
# Ensure all time columns are converted to strings in case they are of type datetime.time
greeter_priority_df['Start_time'] = greeter_priority_df['Start_time'].astype(str)
greeter_priority_df['End_time'] = greeter_priority_df['End_time'].astype(str)
work_status_copy_df['Start_time'] = work_status_copy_df['Start_time'].astype(str)
work_status_copy_df['End_time'] = work_status_copy_df['End_time'].astype(str)

def calculate_remaining_hours(employee, current_time, work_status_df, filtered_df):
    # Convert current_time to datetime object
    current_time = pd.to_datetime(f"{today_date} {current_time}", format="%Y-%m-%d %H:%M:%S")

    # Initialize remaining time
    remaining_time = 0.0

    # Filter for the specific employee's schedule
    employee_schedule = filtered_df[filtered_df['Name'] == employee]
    working_shifts = work_status_df[(work_status_df['Name'] == employee) & (work_status_df['Working Flag'] == 1)]

    #print("employee_schedule\n", employee_schedule)

    # Identify the active shift
    for _, shift in employee_schedule.iterrows():
        shift_start = pd.to_datetime(shift['Time in'])
        shift_end = pd.to_datetime(shift['Time out'])

        #print("\ncurrent_time", current_time)
        #print("shift_end", shift_end, '\n')

        # Skip if the shift has ended
        if current_time >= shift_end:
            #print("current_time >= shift_end")
            continue

        # If within this shift, calculate remaining hours
        if shift_start <= current_time < shift_end:
            #print("shift_start <= current_time < shift_end")
            remaining_time = (shift_end - current_time).total_seconds() / 3600  # Hours
            break
        elif current_time < shift_start:
            #print("current_time < shift_start")
            # If the current time is before the shift, skip to the next one
            continue

    return remaining_time


# Apply the calculation to the DataFrame
greeter_priority_df['Remaining_hours_left'] = greeter_priority_df.apply(
    lambda row: calculate_remaining_hours(row['Name'], row['Start_time'], work_status_copy_df, filtered_df), axis=1
)

# Display the updated DataFrame
greeter_priority_df



,Name,Start_time,End_time,Working Flag,Remaining_hours_left
0,Ana* Gonzalez,08:30:00,09:00:00,1,1.5
1,Ana* Gonzalez,09:00:00,09:30:00,1,1.0
2,Ana* Gonzalez,09:30:00,10:00:00,1,0.5
20,Aidan Priller,09:00:00,09:30:00,1,1.0
21,Aidan Priller,09:30:00,10:00:00,1,0.5
...,...,...,...,...,...
451,Ricardo Gomez,15:30:00,16:00:00,1,1.5
452,Ricardo Gomez,16:00:00,16:30:00,1,1.0
453,Ricardo Gomez,16:30:00,17:00:00,1,0.5
471,Aaron* Dorrance,16:00:00,16:30:00,1,1.0


In [23]:
greeter_priority_df.shape

(165, 5)

In [24]:
greeter_priority_df.head(2)

,Name,Start_time,End_time,Working Flag,Remaining_hours_left
0,Ana* Gonzalez,08:30:00,09:00:00,1,1.5
1,Ana* Gonzalez,09:00:00,09:30:00,1,1.0


In [25]:
work_status_df.shape

(165, 4)

In [26]:
work_status_df.head(2)

,Name,Start_time,End_time,Working Flag
0,Ana* Gonzalez,08:30:00,09:00:00,1
1,Ana* Gonzalez,09:00:00,09:30:00,1


In [27]:
# Testing
greeter_priority_df[greeter_priority_df['Name']=='Analuisa Flores Teran']
#greeter_priority_df[greeter_priority_df['Name']=='Kayhaan Rashiq']
#greeter_priority_df[greeter_priority_df['Name']=='Mali Chavez']
#greeter_priority_df[greeter_priority_df['Name']=='Andrew (AJ) # Bowlen']

,Name,Start_time,End_time,Working Flag,Remaining_hours_left
39,Analuisa Flores Teran,09:00:00,09:30:00,1,2.0
40,Analuisa Flores Teran,09:30:00,10:00:00,1,1.5
41,Analuisa Flores Teran,10:00:00,10:30:00,1,1.0
42,Analuisa Flores Teran,10:30:00,11:00:00,1,0.5
51,Analuisa Flores Teran,15:00:00,15:30:00,1,2.0
52,Analuisa Flores Teran,15:30:00,16:00:00,1,1.5
53,Analuisa Flores Teran,16:00:00,16:30:00,1,1.0
54,Analuisa Flores Teran,16:30:00,17:00:00,1,0.5


In [28]:
greeter_priority_df['Name'].unique()

array(['Ana* Gonzalez', 'Aidan Priller', 'Analuisa Flores Teran',
       'Caroline Lester', 'Carson Turk', 'Ethan Palmer', 'Jackson Carter',
       'Solymar Kneale', 'Teddrose* Tewolde', 'Mariana Cepeda',
       'Marsailles Wesley', 'Om Jadhav', 'Elijah Kodjak', 'Luke* Ruhl',
       'Mohammed Alhazmi', 'Serenity* Munoz', 'Elizabeth Jurry',
       'Summer Sutton', 'Ellia* Bono', 'Mali Chavez', 'Calvin # Levy',
       'Lauren* Koski', 'Mason Bessette', 'Ricardo Gomez',
       'Aaron* Dorrance'], dtype=object)

# Create Shift Requrement

In [29]:
# Import emp_count_req Default Table
# emp_count_req= pd.read_excel(r"00_Input\02_Emp_Count_Requirement.xlsx")
emp_count_req= pd.read_excel('00_Input/02_Emp_Count_Requirement.xlsx')
emp_count_req.head()

# Convert 'From_Time' and 'To_Time' to datetime objects
emp_count_req['From_Time'] = pd.to_datetime(emp_count_req['From_Time'], format='%H:%M:%S').dt.time
emp_count_req['To_Time'] = pd.to_datetime(emp_count_req['To_Time'], format='%H:%M:%S').dt.time

emp_count_req.head()

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed,Greeter_Up_Needed,Greeter_Down_Needed,Min_Total_Emp_Needed
0,07:30:00,08:00:00,0,0,0,0,0
1,08:00:00,08:30:00,0,0,0,0,0
2,08:30:00,09:00:00,0,0,0,0,0
3,09:00:00,09:30:00,3,3,1,0,7
4,09:30:00,10:00:00,3,3,1,0,7


In [30]:
work_status_df.head()

,Name,Start_time,End_time,Working Flag
0,Ana* Gonzalez,08:30:00,09:00:00,1
1,Ana* Gonzalez,09:00:00,09:30:00,1
2,Ana* Gonzalez,09:30:00,10:00:00,1
20,Aidan Priller,09:00:00,09:30:00,1
21,Aidan Priller,09:30:00,10:00:00,1


In [31]:
# Check if available employees are sufficient to satisfy the required count

emp_aval= work_status_df.copy()
emp_aval.rename(columns={'Start_time': 'Work_From', 'End_time': 'Work_To', 'Working Flag': 'Working_Flag'}, inplace=True)

# Create "total available employee" column

# Group By the From and To time of "Work Status Per Time" table and count the number of available employees
grouped_avl_by_time= emp_aval.groupby(['Work_From', 'Work_To']).agg({'Working_Flag': 'sum'}).reset_index()
grouped_avl_by_time.columns= ['Work_From', 'Work_To', 'Total_Avl_Emp']

# Join it with the emp_count_req table
emp_demand_check= pd.merge(emp_count_req, grouped_avl_by_time, how='left', left_on=['From_Time','To_Time'], right_on=['Work_From','Work_To'])
columns_needed= ['From_Time', 'To_Time', 'Reg_Up_Needed', 'Reg_Down_Needed', 'Greeter_Up_Needed', 'Greeter_Down_Needed', 'Min_Total_Emp_Needed', 'Total_Avl_Emp']
emp_demand_check= emp_demand_check[columns_needed]
emp_demand_check['Total_Avl_Emp'].fillna(0, inplace=True) # If no emp are available on a selected shift, make the availability zero. 

# Create an availability check flag and alert 
def alert_employee_shortage(emp_demand_check: pd.DataFrame):
    shortage= emp_demand_check[emp_demand_check['Availability_Check_Flag'] == False]
    if(len(shortage)==0):
        print("No shortage of employees for the whole day")
    else:
        print("ALERT: Employees are on shortage for the following time slots")
        print(shortage[['From_Time', 'To_Time', 'Min_Total_Emp_Needed', 'Total_Avl_Emp']])
    return

emp_demand_check['Availability_Check_Flag']= emp_demand_check['Min_Total_Emp_Needed']<= emp_demand_check['Total_Avl_Emp']
alert_employee_shortage(emp_demand_check)
display(emp_demand_check.sample(5))


No shortage of employees for the whole day


,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed,Greeter_Up_Needed,Greeter_Down_Needed,Min_Total_Emp_Needed,Total_Avl_Emp,Availability_Check_Flag
22,18:30:00,19:00:00,0,0,0,0,0,0.0,True
3,09:00:00,09:30:00,3,3,1,0,7,9.0,True
4,09:30:00,10:00:00,3,3,1,0,7,9.0,True
9,12:00:00,12:30:00,3,3,1,1,8,11.0,True
17,16:00:00,16:30:00,3,3,1,1,8,10.0,True


# Greeter Assignment

In [32]:
greeter_assignment=emp_demand_check[['From_Time','To_Time','Greeter_Down_Needed','Greeter_Up_Needed']]
greeter_assignment.head()

,From_Time,To_Time,Greeter_Down_Needed,Greeter_Up_Needed
0,07:30:00,08:00:00,0,0
1,08:00:00,08:30:00,0,0
2,08:30:00,09:00:00,0,0
3,09:00:00,09:30:00,0,1
4,09:30:00,10:00:00,0,1


In [33]:
# Create greeter shift completed dictionary

#  Get unique names
unique_names = filtered_df['Name'].unique()

# Create a dictionary with names as keys and 0 as the initial value
greeter_shift_done_dict = {name: 0 for name in unique_names}

# Display the initialized dictionary
greeter_shift_done_dict


{'Ana* Gonzalez': 0,
 'Aidan Priller': 0,
 'Analuisa Flores Teran': 0,
 'Caroline Lester': 0,
 'Carson Turk': 0,
 'Ethan Palmer': 0,
 'Jackson Carter': 0,
 'Solymar Kneale': 0,
 'Teddrose* Tewolde': 0,
 'Mariana Cepeda': 0,
 'Marsailles Wesley': 0,
 'Om Jadhav': 0,
 'Elijah Kodjak': 0,
 'Luke* Ruhl': 0,
 'Mohammed Alhazmi': 0,
 'Serenity* Munoz': 0,
 'Elizabeth Jurry': 0,
 'Summer Sutton': 0,
 'Ellia* Bono': 0,
 'Mali Chavez': 0,
 'Calvin # Levy': 0,
 'Lauren* Koski': 0,
 'Mason Bessette': 0,
 'Ricardo Gomez': 0,
 'Aaron* Dorrance': 0}

In [34]:
# Convert 'Start_time' and 'End_time' to time objects in the main dataframe
greeter_priority_df['Start_time'] = pd.to_datetime(greeter_priority_df['Start_time'], format='%H:%M:%S').dt.time
greeter_priority_df['End_time'] = pd.to_datetime(greeter_priority_df['End_time'], format='%H:%M:%S').dt.time

def assign_priority(df, filtered_df, greeter_shift_done_dict):
    # Initialize the priority column
    df['Priority'] = 0  # Initialize the Priority column with zeros
    
    # Group by Start_time and End_time
    for (start, end), group in df.groupby(['Start_time', 'End_time']):
        responsible_people = filtered_df[(filtered_df['Time in'] == start) & 
                                         (filtered_df['Time out'] == end) & 
                                         (filtered_df['Responsibility'] == 'Greeter')]
        
        if not responsible_people.empty:
            for index, row in responsible_people.iterrows():
                greeter = row['Name']
                if greeter in group['Name'].values:
                    df.loc[(df['Start_time'] == start) & (df['End_time'] == end) & (df['Name'] == greeter), 'Priority'] = 1
            
            remaining_group = group[~group['Name'].isin(responsible_people['Name'])]
            non_zero_hours_group = remaining_group[remaining_group['Remaining_hours_left'] > 0].sort_values(by='Remaining_hours_left', ascending=True)
            
            # Assign dense ranking starting from 2 for non-zero hours employees
            if not non_zero_hours_group.empty:
                non_zero_hours_group['Priority'] = non_zero_hours_group['Remaining_hours_left'].rank(method='dense', ascending=True).astype(int) + 1
                df.loc[non_zero_hours_group.index, 'Priority'] = non_zero_hours_group['Priority']
            
            # Employees with 0 hours remaining
            zero_hours_group = remaining_group[remaining_group['Remaining_hours_left'] == 0]
            if not zero_hours_group.empty:
                max_priority = non_zero_hours_group['Priority'].max() if not non_zero_hours_group.empty else 1
                zero_hours_group['Priority'] = max_priority + 1
                df.loc[zero_hours_group.index, 'Priority'] = zero_hours_group['Priority']
        else:
            group_with_non_zero_hours = group[group['Remaining_hours_left'] > 0].sort_values(by='Remaining_hours_left', ascending=True)
            group_with_zero_hours = group[group['Remaining_hours_left'] == 0]
            if not group_with_non_zero_hours.empty:
                group_with_non_zero_hours['Priority'] = group_with_non_zero_hours['Remaining_hours_left'].rank(method='dense', ascending=True).astype(int)
                df.loc[group_with_non_zero_hours.index, 'Priority'] = group_with_non_zero_hours['Priority']
            if not group_with_zero_hours.empty:
                max_priority = group_with_non_zero_hours['Priority'].max() if not group_with_non_zero_hours.empty else 1
                group_with_zero_hours['Priority'] = max_priority + 1
                df.loc[group_with_zero_hours.index, 'Priority'] = group_with_zero_hours['Priority']

    return df  # Return the modified DataFrame with the Priority column

In [35]:
def process_time_periods(greeter_priority_df, greeter_assignment, filtered_df, greeter_shift_done_dict):
    
    # Assign priorities before processing time periods
    greeter_priority_df = assign_priority(greeter_priority_df, filtered_df, greeter_shift_done_dict)

    time_periods = greeter_assignment[['From_Time', 'To_Time', 'Greeter_Up_Needed', 'Greeter_Down_Needed']].drop_duplicates()

    for idx, period in time_periods.iterrows():
        # Extract needed counts for upstairs and downstairs greeters
        up_needed = int(period['Greeter_Up_Needed'])
        down_needed = int(period['Greeter_Down_Needed'])

        current_period_data = greeter_priority_df[
            (greeter_priority_df['Start_time'] == period['From_Time']) &
            (greeter_priority_df['End_time'] == period['To_Time'])
        ]

        # Debug: Check current period data
        print(f"Current Period: {period['From_Time']} to {period['To_Time']}")
        print("Current Period Data:\n", current_period_data)

        # Sort employees by priority
        sorted_employees = current_period_data.sort_values('Priority')

        # Assign upstairs greeters based on needed count
        upstairs_greeters = sorted_employees.head(up_needed)['Name'].tolist()
        print("Upstairs Greeters Assigned:", upstairs_greeters)

        # Filter out already assigned upstairs greeters for downstairs assignments
        remaining_employees = sorted_employees[~sorted_employees['Name'].isin(upstairs_greeters)]

        # Assign downstairs greeters based on needed count
        downstairs_greeters = remaining_employees.head(down_needed)['Name'].tolist() if down_needed > 0 else []
        print("Downstairs Greeters Assigned:", downstairs_greeters)
        print("-----------------------------------------------------------------------------")

        # Update the greeter_assignment DataFrame with assigned greeters
        greeter_assignment.at[idx, 'Upstairs Greeter'] = upstairs_greeters[0] if upstairs_greeters else None
        greeter_assignment.at[idx, 'Downstairs Greeter'] = downstairs_greeters[0] if downstairs_greeters else None

        # Update shift counts for assigned employees
        for employee in upstairs_greeters + downstairs_greeters:
            if employee:  # Only update for actual assignments
                greeter_shift_done_dict[employee] += 1

        # Update the priorities for assigned greeters
        for employee in upstairs_greeters + downstairs_greeters:
            if employee:  # Only update for actual assignments
                greeter_priority_df.loc[greeter_priority_df['Name'] == employee, 'Priority'] = greeter_priority_df['Priority'].max() + 1

    return greeter_assignment, greeter_shift_done_dict

# Example call to the function
greeter_assignment, greeter_shift_done_dict = process_time_periods(
    greeter_priority_df, 
    greeter_assignment, 
    filtered_df, 
    greeter_shift_done_dict
)

# Output results
greeter_assignment.head(50)

Current Period: 07:30:00 to 08:00:00
Current Period Data:
 Empty DataFrame
Columns: [Name, Start_time, End_time, Working Flag, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:00:00 to 08:30:00
Current Period Data:
 Empty DataFrame
Columns: [Name, Start_time, End_time, Working Flag, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:30:00 to 09:00:00
Current Period Data:
             Name Start_time  End_time  Working Flag  Remaining_hours_left  \
0  Ana* Gonzalez   08:30:00  09:00:00             1                   1.5   

   Priority  
0         1  
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
------------------------------------------------------------------------

,From_Time,To_Time,Greeter_Down_Needed,Greeter_Up_Needed,Upstairs Greeter,Downstairs Greeter
0,07:30:00,08:00:00,0,0,None,None
1,08:00:00,08:30:00,0,0,None,None
2,08:30:00,09:00:00,0,0,None,None
3,09:00:00,09:30:00,0,1,Ana* Gonzalez,None
4,09:30:00,10:00:00,0,1,Aidan Priller,None
5,10:00:00,10:30:00,0,1,Analuisa Flores Teran,None
6,10:30:00,11:00:00,0,1,Jackson Carter,None
7,11:00:00,11:30:00,1,1,Caroline Lester,Serenity* Munoz
8,11:30:00,12:00:00,1,1,Carson Turk,Ethan Palmer
9,12:00:00,12:30:00,1,1,Marsailles Wesley,Luke* Ruhl


# Register Allocation

In [36]:
filtered_df.head(2)

,Name,Responsibility,Time in,Time out
0,Ana* Gonzalez,Lead Student,2024-12-09 08:30:00,2024-12-09 10:00:00
1,Aidan Priller,Sales Floor,2024-12-09 09:00:00,2024-12-09 10:00:00


In [40]:
len(filtered_df['Name'].unique())

25

In [37]:
work_status_df.head(2)

,Name,Start_time,End_time,Working Flag
0,Ana* Gonzalez,08:30:00,09:00:00,1
1,Ana* Gonzalez,09:00:00,09:30:00,1


In [42]:
len(work_status_df['Name'].unique())

25

In [38]:
emp_demand_check.head(2)

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed,Greeter_Up_Needed,Greeter_Down_Needed,Min_Total_Emp_Needed,Total_Avl_Emp,Availability_Check_Flag
0,07:30:00,08:00:00,0,0,0,0,0,0.0,True
1,08:00:00,08:30:00,0,0,0,0,0,0.0,True


In [17]:
# Collect register needed count

shift_req_df=emp_demand_check[['From_Time', 'To_Time', 'Reg_Up_Needed', 'Reg_Down_Needed']]
shift_req_df.head()

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed
0,07:30:00,08:00:00,0,0
1,08:00:00,08:30:00,0,0
2,08:30:00,09:00:00,0,0
3,09:00:00,09:30:00,3,3
4,09:30:00,10:00:00,3,3


In [18]:
# Collect emp's work status

work_status_df= greeter_priority_df[greeter_priority_df['Working Flag']==1][['Name', 'Start_time', 'End_time', 'Working Flag', 'Remaining_hours_left']]
work_status_df.sample(2)

,Name,Start_time,End_time,Working Flag,Remaining_hours_left
330,Summer Sutton,12:00:00,12:30:00,1,1.0
115,Jackson Carter,09:00:00,09:30:00,1,2.0


In [ ]:
# Allocate Registers

register_allocation= shift_req_df.copy()
register_allocation['Register Up']= None
register_allocation['Register Down']= None
register_allocation['SF Up']= None
register_allocation['SF Down']= None

# Prepare intial variables
current_RUs= []
current_RDs= []
store_open_time= shift_req_df['From_Time'].min() #time(9, 00) 
store_close_time= shift_req_df['From_Time'].max()
time_delta= timedelta(minutes=30)

curr_time= store_open_time

while (curr_time < store_close_time):
    curr_time_plus30= (datetime.combine(datetime.today(), curr_time) + time_delta).time()
    print("---------------------------------------------------------------------------------------------")
    print(f"Allocating registers between {curr_time} to {curr_time_plus30} ...")

    # Find how many total registers are needed
    needed_RU_count, needed_RD_count= shift_req_df[(shift_req_df['From_Time']== curr_time) & (shift_req_df['To_Time']== curr_time_plus30)][['Reg_Up_Needed','Reg_Down_Needed']].values[0]
    if (needed_RU_count + needed_RD_count == 0):
        print("No Registers Needed")
        curr_time= curr_time_plus30
        continue

    # Working Employees
    all_working_emp_list= work_status_df[(work_status_df['Start_time']== curr_time) & (work_status_df['End_time']== curr_time_plus30)]['Name'].to_list()

    # Greeters allocated already for the current shift
    greeter_filtered= greeter_assignment[(greeter_assignment['From_Time']== curr_time) & (greeter_assignment['To_Time']== curr_time_plus30)]
    greeter_emp_list= [greeter_filtered['Upstairs Greeter'].values[0],greeter_filtered['Downstairs Greeter'].values[0]]
    if None in greeter_emp_list:
        greeter_emp_list.remove(None)
    greeter_emp_list

    # Baseed on the workring and greeter list, calculate the retined register list
    retained_RUs=[]
    retained_RDs= []

    for emp in current_RUs:
        if (emp not in all_working_emp_list):
            # EMP shift got over
            continue
        if (emp in greeter_emp_list):
            # EMP moved to greeter
            continue
        else:
            retained_RUs.append(emp)
    
    for emp in current_RDs:
        if (emp not in all_working_emp_list):
            # EMP shift got over
            continue
        if (emp in greeter_emp_list):
            # EMP moved to greeter
            continue
        else:
            retained_RDs.append(emp)

    # Find how many new registers are needed
    RU_retained_count= len(retained_RUs)
    RD_retained_count= len(retained_RDs)
    new_RU_needed_count= needed_RU_count-RU_retained_count
    new_RD_needed_count= needed_RD_count-RD_retained_count

    # Create the priority table. Use the same table to assign both RU and RD
    priority_table= work_status_df[(work_status_df['Start_time']== curr_time) & (work_status_df['End_time']== curr_time_plus30) & (work_status_df['Name'] not in retained_RUs+retained_RDs)]
    priority_table['RU_Priority'] = priority_table['Remaining_hours_left'].rank(method='min', ascending=False).astype(int)
    priority_table.sort_values(by='RU_Priority', inplace=True)
    priority_table.reset_index(drop=True, inplace=True)
    display(priority_table.head())

    # Pick the new registers for both RU and RD
    print('RU_retained_count', RU_retained_count)
    print('RD_retained_count', new_RU_needed_count)
    print('new_RU_needed_count', new_RU_needed_count)
    print('new_RD_needed_count', new_RD_needed_count)
    new_RU_assigned= priority_table['Name'][:new_RU_needed_count].tolist()
    new_RD_assigned= priority_table['Name'][new_RU_needed_count: new_RU_needed_count+new_RD_needed_count].tolist()
    print('retained_RUs', retained_RUs)
    print('retained_RDs', retained_RDs)
    print('new_RU_assigned', new_RU_assigned)
    print('new_RD_assigned', new_RD_assigned)

    # Assign the remaining to salesfloor
    unassigned_count= len(priority_table)-(new_RU_needed_count+new_RD_needed_count)
    if(unassigned_count<=0):
        salesfloor_up_assigned_count=0
        salesfloor_up_assigned= []
        salesfloor_dwn_assigned_count=0
        salesfloor_dwn_assigned= []
    elif(unassigned_count==1):
        salesfloor_up_assigned_count=1
        salesfloor_up_assigned= priority_table['Name'][-1].tolist()
        salesfloor_dwn_assigned_count=0
        salesfloor_dwn_assigned= []
    else:
        salesfloor_up_assigned_count=int(unassigned_count/2)
        salesfloor_up_assigned= priority_table['Name'][new_RU_needed_count+new_RD_needed_count:new_RU_needed_count+new_RD_needed_count+salesfloor_up_assigned_count].tolist()
        salesfloor_dwn_assigned_count=unassigned_count-salesfloor_up_assigned_count
        salesfloor_dwn_assigned= priority_table['Name'][new_RU_needed_count+new_RD_needed_count+salesfloor_up_assigned_count:].tolist()
    print('unassigned_count', unassigned_count)
    print('salesfloor_up_assigned_count', salesfloor_up_assigned_count)
    print('salesfloor_up_assigned', salesfloor_up_assigned)
    print('salesfloor_dwn_assigned_count', salesfloor_dwn_assigned_count)
    print('salesfloor_dwn_assigned', salesfloor_dwn_assigned)
    
    # Update the registers in the final table
    for i, row in register_allocation.iterrows():
        if row['From_Time'] == curr_time and row['To_Time'] == curr_time_plus30:
            # Assign new register up (RU) employees
            register_allocation.at[i, 'Register Up'] = retained_RUs+new_RU_assigned
            # Assign new register down (RD) employees
            register_allocation.at[i, 'Register Down'] = retained_RDs+new_RD_assigned
            # Assign new SF up employees
            register_allocation.at[i, 'SF Up'] = salesfloor_up_assigned
            # Assign new SF down employees
            register_allocation.at[i, 'SF Down'] = salesfloor_dwn_assigned
    
    # Should not break this, rather increment the start time by 30mins so that the while loop continues
    #break
    curr_time= curr_time_plus30

register_allocation

In [27]:
emp_demand_check.head()

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed,Greeter_Up_Needed,Greeter_Down_Needed,Min_Total_Emp_Needed,Total_Avl_Emp,Availability_Check_Flag
0,07:30:00,08:00:00,0,0,0,0,0,0.0,True
1,08:00:00,08:30:00,0,0,0,0,0,0.0,True
2,08:30:00,09:00:00,0,0,0,0,0,1.0,True
3,09:00:00,09:30:00,3,3,1,0,7,9.0,True
4,09:30:00,10:00:00,3,3,1,0,7,9.0,True


In [28]:
shift_req_df.head()

,From_Time,To_Time,Reg_Up_Needed,Reg_Down_Needed
0,07:30:00,08:00:00,0,0
1,08:00:00,08:30:00,0,0
2,08:30:00,09:00:00,0,0
3,09:00:00,09:30:00,3,3
4,09:30:00,10:00:00,3,3


In [29]:
final_allocation= pd.merge(greeter_assignment, register_allocation, how='outer', left_on=['From_Time', 'To_Time'], right_on=['From_Time', 'To_Time'])
final_allocation.sample(3)

,From_Time,To_Time,Greeter_Down_Needed,Greeter_Up_Needed,Upstairs Greeter,Downstairs Greeter,Reg_Up_Needed,Reg_Down_Needed,Register Up,Register Down,SF Up,SF Down
16,15:30:00,16:00:00,1,1,Lauren* Koski,Ricardo Gomez,3,3,"[Analuisa Flores Teran, Jackson Carter, Marian...","[Luke* Ruhl, Elizabeth Jurry, Ellia* Bono]","[Lauren* Koski, Ricardo Gomez]","[Mohammed Alhazmi, Calvin # Levy, Mason Bessette]"
11,13:00:00,13:30:00,1,1,Om Jadhav,Elijah Kodjak,3,3,"[Jackson Carter, Mariana Cepeda, Elizabeth Jurry]","[Ellia* Bono, Mohammed Alhazmi, Aidan Priller]","[Solymar Kneale, Om Jadhav]","[Elijah Kodjak, Serenity* Munoz, Mali Chavez]"
22,18:30:00,19:00:00,0,0,None,None,0,0,None,None,None,None


In [42]:
final_allocation.to_excel(f"01_Output\Final_Allocation_{datetime.now().strftime('%m-%y-%d_%H-%M-%S')}.xlsx", index=True)

In [ ]:
# TO DO

#- Fix time remaining calculation logic. 
# 	- Person doing multiple shifts with breaks in between not working.
# 	- Time remaining should be from the current time.

# - Test with matching emp count req and emp availability

# - Currently ignoring if "Greeter" as initial role assigned 
# - Take the reamining available emp not in both greeter and register and allocate them to salesfloor
# - Seperate UP and DOWN logic
# - Is it okay to allocate SF as all the remaining unallocated?
# - We won't have any time slot without registers but SF right?

# CONSIDERATIONS

# - Remember the shifts should not end in 15mins interval (unless it's 6:15 if the store closes at 6)
# - Currently if the initial shift allocation starts at any 15mins interval (like 8:45), we are shifting to later 30mins (9:00) AND if the initial shift allocation ends at any 15mins interval (like 6:15), we are shifting to previous 30mins (6:00). This can be avoided if the time inteval is set to every 15mins instead of 30mins.